<a href="https://colab.research.google.com/github/timsetsfire/agents-on-colab/blob/main/MLOps_Agents_(on_Trial).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## MLOps Agent

## The scope of this Notebook is to provide instructions on how to use DataRobot's MLOps Agents.


resources:

* Agents webinar: https://community.datarobot.com/t5/learning-sessions/monitoring-all-your-models-with-datarobot-agents/ba-p/7732
* Working with Remote Models: https://community.datarobot.com/t5/resources/working-with-remote-models/ta-p/7517
* Deploy in SageMaker and Monitor with MLOps Agents: https://community.datarobot.com/t5/resources/deploy-in-sagemaker-and-monitor-with-mlops-agents/ta-p/5771


In [1]:
%%sh
git clone https://github.com/timsetsfire/agents-plus-challengers.git

Cloning into 'agents-plus-challengers'...


In [13]:
%%sh
pip install -r /content/agents-plus-challengers/remote-model/requirements.txt -q
pip install datarobot urllib3==1.26.4 -U -q
pip install datarobot-mlops -q
pip install datarobot-mlops-connected-client -q

In [3]:
import pandas as pd
import bson
import time
import sys
import pprint
import os
import datarobot as dr
import yaml
import subprocess 
import urllib3
import requests
import json
import bson
sys.path.append("/content/agents-plus-challengers/remote-model")

In [4]:
df = pd.read_csv(
    "/content/agents-plus-challengers/data/DR_Demo_SCMS.csv"
    ).sample(frac=0.3)
target = "Late_delivery"
df[target].describe()

df["AssociationID"] = 1
df["AssociationID"] = df["AssociationID"].apply(lambda x: str(bson.ObjectId()))

sampled_df = df.sample(frac = 0.20)

## Grab agents tarball from DataRobot

In [5]:
token = "token"
endpoint = "https://app2.datarobot.com"
## connect to DataRobot platform with datarobot python client. 
client = dr.Client(token, "{}/api/v2".format(endpoint))
## grab mlops agents tarball
mlops_agents_tb = client.get("mlopsInstaller")
with open("/content/mlops-agent.tar.gz", "wb") as f:
    f.write(mlops_agents_tb.content)
## unpack tarball
os.system("tar -xf /content/mlops-agent.tar.gz -C . && mkdir -p /tmp/ta")

0

## Agents Framework

1. Scoring pipeline - used to score data
2. Python MLOps library - used to relay data to a buffer.  Buffer could be any of 
  * file system - used in this example
  * pubsub
  * kafka
  * sqs
  * rabbit
2. Tracking Agents (light weight java service) - monitors the buffer and relays data to DR MLOps


### Configure Tracking Agents

Very vanilla configuration


Using File system as a buffer

In [6]:
import glob
agents_dir = glob.glob("/content/datarobot_mlops*").pop()
print(agents_dir)
with open('{}/conf/mlops.agent.conf.yaml'.format(agents_dir)) as file:
    documents = yaml.load(file)
## configure the loaction of the mlops instance with which we'll communcate
documents['mlopsUrl'] = endpoint
# Set your API token
documents['apiToken'] = token
## write the configuration back to disk
with open('{}/conf/mlops.agent.conf.yaml'.format(agents_dir), "w") as f:
    yaml.dump(documents, f)
## start the tracking agents service
subprocess.call("{}/bin/start-agent.sh".format(agents_dir))
## check status of agents service
check = subprocess.Popen(["{}/bin/status-agent.sh".format(agents_dir)], stdout=subprocess.PIPE)
print(check.stdout.readlines())
check.terminate()

/content/datarobot_mlops_package-8.0.3
[b'DataRobot MLOps-Agent is running as a service.\n']


In [11]:
os.system('pip install {}/lib/datarobot_mlops-*.whl'.format(agents_dir))

0

## Upload Training Data

In [7]:
print("Uploading training data. This may take some time...")
training_data = dr.Dataset.create_from_in_memory_data(df.drop(["AssociationID"], axis=1))

Uploading training data. This may take some time...


## Register Model, and Enable Monitoring with DataRobot MLOps

In [15]:
from datarobot.mlops.mlops import MLOps
from datarobot.mlops.connected.client import MLOpsClient


model_info = {
              "name": "Late shipment model",
              "modelDescription": {
                "description": "Binary classification on late shipment dataset",
                "location": "colab"
              },
              "target": {
                 "type": "Binary",
                 "name": "Late_delivery",
                 "classNames": ["1","0"],  # minority/positive class should be listed first
                 "predictionThreshold": 0.5
                }
            }

DEPLOYMENT_NAME="Google Colab - Python binary classificationnew"

model_info["datasets"] = {"trainingDataCatalogId": training_data.id}


# Create connected client
mlops_connected_client = MLOpsClient(endpoint, token)
# Create the model package
print('Create model package')
model_pkg_id = mlops_connected_client.create_model_package(model_info)
model_pkg = mlops_connected_client.get_model_package(model_pkg_id)
model_id = model_pkg["modelId"]
# Deploy the model package
print('Deploy model package')
deployment_id = mlops_connected_client.deploy_model_package(model_pkg["id"], label = DEPLOYMENT_NAME)

Create model package
Deploy model package


## Update Drift Tracking

In [16]:
deployment = dr.Deployment.get(deployment_id)
deployment.update_drift_tracking_settings(target_drift_enabled=True, feature_drift_enabled=True)

## Update accuracy tracking

In [17]:
deployment.update_association_id_settings(column_names = ["AssociationID"], required_in_prediction_requests=False)

## Review deployment detail

In [18]:
DEPLOYMENT_ID = deployment.id
MODEL_ID = deployment.model["id"]

deployment_details = client.get(f"deployments/{deployment.id}").json()

deployment_details_df = pd.DataFrame({"model name": deployment_details["label"],
 "model description": deployment_details["description"],
 "create date": deployment_details["createdAt"],
 "deployment id": deployment_details["id"], 
 "approval status": deployment_details["approvalStatus"], 
 "importance": deployment_details["importance"],
 "owners": deployment_details["owners"]}).T
 
deployment_details_df[["preview"]]

,preview
model name,Google Colab - Python binary classificationnew
model description,
create date,2021-12-15T14:33:30.306000Z
deployment id,61b9fcba9c4f384b9cf4f5b9
approval status,APPROVED
importance,None
owners,"[{'id': '5f6b61165c439d2e6e6d10c3', 'firstName..."


## Original Scoring Pipepline

In [19]:
model = pd.read_pickle("/content/agents-plus-challengers/remote-model/artifact.pkl")
predictions = model.predict_proba(sampled_df.drop([target, "AssociationID"], axis=1)).values.tolist()
predictions[0:5]

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:439: UserWarning: X has feature names, but SimpleImputer was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


[[0.993935910819661, 0.006064089180338972],
 [0.6715751461951487, 0.3284248538048513],
 [0.9817904103020143, 0.01820958969798564],
 [0.9230363252091739, 0.07696367479082611],
 [0.5536388096636268, 0.4463611903363732]]

## Modified Scoring Pipeline

In [20]:
from datarobot.mlops.mlops import MLOps
# setting environment variables 
os.environ["MLOPS_DEPLOYMENT_ID"]= DEPLOYMENT_ID
os.environ["MLOPS_MODEL_ID"]= MODEL_ID
os.environ["MLOPS_SPOOLER_TYPE"]= "FILESYSTEM"
os.environ["MLOPS_FILESYSTEM_DIRECTORY"]= "/tmp/ta"
# init mlops client
mlops = MLOps().init()
# Load model
model = pd.read_pickle("/content/agents-plus-challengers/remote-model/artifact.pkl")
# start timeing
start_time = time.time()
############## original scoring pipeline ##############
predictions = model.predict_proba(sampled_df.drop([target, "AssociationID"],axis=1)).values.tolist()
#######################################################
# end timer
end_time = time.time()
# number of predictions
num_predictions = len(predictions)
# report stats
mlops.report_deployment_stats(num_predictions, end_time - start_time)
# report the predictions data: features, predictions
mlops.report_predictions_data(features_df=sampled_df.drop([target], axis=1), 
                                predictions=predictions, class_names = ["0", "1"])
# shutdown python client.  Java service is still running
mlops.shutdown()

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:439: UserWarning: X has feature names, but SimpleImputer was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


In [21]:
check = subprocess.Popen(["{}/bin/status-agent.sh".format(agents_dir)], stdout=subprocess.PIPE)
print(check.stdout.readlines())
check.terminate()

[b'DataRobot MLOps-Agent is running as a service.\n']


## Submitting Actuals

In [22]:
actuals_for_submission = sampled_df[[target, "AssociationID"]].rename(columns = {target: "actual_value", "AssociationID": "association_id"})
deployment.submit_actuals(actuals_for_submission)

In [23]:
print("https://app2.datarobot.com/deployments/{}/overview".format(os.environ["MLOPS_DEPLOYMENT_ID"]))

https://app2.datarobot.com/deployments/61b9fcba9c4f384b9cf4f5b9/overview


In [24]:
service_stats = deployment.get_service_stats()
service_stats.metrics
pd.DataFrame([service_stats.metrics]).T

,0
totalPredictions,619.000000
totalRequests,1.000000
slowRequests,0.000000
executionTime,0.014383
responseTime,0.000000
userErrorRate,0.000000
serverErrorRate,0.000000
numConsumers,1.000000
cacheHitRatio,0.000000
medianLoad,0.000000


## Retrieve Accuracy metrics

In [25]:
accuracy = deployment.get_accuracy_over_time()
metrics = ['AUC', 'FVE Binomial', 'Gini Norm', 'Kolmogorov-Smirnov', 'LogLoss', 'MCC', 'PPV', 'NPV', 'TPR', 'FPR', 'F1','Rate@Top10%', 'Rate@Top5%']
metrics_ls = []
for metric in metrics:
    m = deployment.get_accuracy_over_time(metric = metric)
    start = m.summary["period"]["start"]
    end = m.summary["period"]["end"]
    temp = [metric, m.baseline["value"], m.summary["value"], start, end, m.summary["sample_size"]]
    metrics_ls.append(temp)     
metrics_df = pd.DataFrame(metrics_ls, columns = ["metric", "baseline", "value", "start", "end", "n"])
metrics_df

,metric,baseline,value,start,end,n
0,AUC,None,None,2021-12-08 15:00:00+00:00,2021-12-15 15:00:00+00:00,None
1,FVE Binomial,None,None,2021-12-08 15:00:00+00:00,2021-12-15 15:00:00+00:00,None
2,Gini Norm,None,None,2021-12-08 15:00:00+00:00,2021-12-15 15:00:00+00:00,None
3,Kolmogorov-Smirnov,None,None,2021-12-08 15:00:00+00:00,2021-12-15 15:00:00+00:00,None
4,LogLoss,None,None,2021-12-08 15:00:00+00:00,2021-12-15 15:00:00+00:00,None
5,MCC,None,None,2021-12-08 15:00:00+00:00,2021-12-15 15:00:00+00:00,None
6,PPV,None,None,2021-12-08 15:00:00+00:00,2021-12-15 15:00:00+00:00,None
7,NPV,None,None,2021-12-08 15:00:00+00:00,2021-12-15 15:00:00+00:00,None
8,TPR,None,None,2021-12-08 15:00:00+00:00,2021-12-15 15:00:00+00:00,None
9,FPR,None,None,2021-12-08 15:00:00+00:00,2021-12-15 15:00:00+00:00,None


## Retrieve Drift Metrics

In [26]:
feature_drift = deployment.get_feature_drift()
fd_ls = [[fd.name, fd.metric, fd.drift_score, fd.feature_impact, fd.period["start"], fd.period["end"]] for fd in feature_drift]
drift_df = pd.DataFrame(fd_ls, columns = ["feature", "drift_metric", "drift_score", "feature Importance", "start", "end"])
drift_df

,feature,drift_metric,drift_score,feature Importance,start,end
0,ID,psi,None,1.000000,2021-12-08 15:00:00+00:00,2021-12-15 15:00:00+00:00
1,Vendor,psi,None,0.603024,2021-12-08 15:00:00+00:00,2021-12-15 15:00:00+00:00
2,Vendor INCO Term,psi,None,0.447592,2021-12-08 15:00:00+00:00,2021-12-15 15:00:00+00:00
3,Fulfill Via,psi,None,0.424402,2021-12-08 15:00:00+00:00,2021-12-15 15:00:00+00:00
4,Item Description,psi,None,0.372512,2021-12-08 15:00:00+00:00,2021-12-15 15:00:00+00:00
5,Pack Price,psi,None,0.309771,2021-12-08 15:00:00+00:00,2021-12-15 15:00:00+00:00
6,Country,psi,None,0.270832,2021-12-08 15:00:00+00:00,2021-12-15 15:00:00+00:00
7,Manufacturing Site,psi,None,0.262858,2021-12-08 15:00:00+00:00,2021-12-15 15:00:00+00:00
8,Molecule/Test Type,psi,None,0.260338,2021-12-08 15:00:00+00:00,2021-12-15 15:00:00+00:00
9,Brand,psi,None,0.220255,2021-12-08 15:00:00+00:00,2021-12-15 15:00:00+00:00
